# Molecular similarity network with displayed structures
Code adapted from [Draw molecular network on Jupyter notebook with rdkit and cytoscape.js-2 #RDKit #cytoscape](https://iwatobipen.wordpress.com/2019/03/20/draw-molecular-network-on-jupyter-notebook-with-rdkit-and-cytoscape-js-2-rdkit-cytoscape/)  

Activate RDkit environment and enable the cyjupyter extension before running Jupyter notebook:
```bash
$ source activate my-rdkit-env
$ jupyter nbextension enable --py --sys-prefix cyjupyter
```

In [115]:
import networkx as nx
from networkx.readwrite import cytoscape_data
from rdkit import Chem
from rdkit.Chem import RDConfig
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from cyjupyter import Cytoscape
import cyjupyter
import os
from urllib import parse

In [117]:
# handle image conversion
def smi2svg(smi):
    mol = Chem.MolFromSmiles(smi)
    try:
        Chem.rdmolops.Kekulize(mol)
    except:
        pass
    drawer = rdMolDraw2D.MolDraw2DSVG(400, 200)
    AllChem.Compute2DCoords(mol)
    drawer.DrawMolecule(mol)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText().replace("svg:", "")
    return svg

def smi2image(smi):
    svg_string = smi2svg(smi)
    impath = 'data:image/svg+xml;charset=utf-8,' + parse.quote(svg_string, safe="")
    return impath

In [116]:
# load SMILES file 
# format: 1st column SMILES | 2nd column compound name
smis, nams = [], []
with open("/home/yossa/notebook/cb/gtp+xtal.smi", 'r') as smi_nam_list:
    for smi_nam in smi_nam_list:
        smis.append(smi_nam.rstrip().split(' ')[0])
        nams.append(smi_nam.rstrip().split(' ')[1])

print(str(len(smis))+" molecules loaded from SMILES file")

# print(smis)
# print(nams)

23 molecules loaded from SMILES file


In [118]:
g = nx.Graph()

# add node
for smi,nam in zip(smis,nams):
    #print(smi,nam)
    g.add_node(smi, img=smi2image(smi), name=nam)

# add edge if Tc >= threshold
mols = [Chem.MolFromSmiles(x) for x in smis]
fps = [AllChem.GetMorganFingerprintAsBitVect(x,2,1024) for x in mols]
for i in range(len(smis)):
    for j in range(i):
        Tc = DataStructs.TanimotoSimilarity(fps[i], fps[j])
        if Tc >= 0.3:
            g.add_edge(smis[i], smis[j])

# style
stobj=[
  {'style': [{'css': {
      'background-color': 'blue',
      'shape' : 'rectangle',
      'width':800,
      'height':400,
      'border-color': 'rgb(0,0,0)',
      'border-opacity': 1.0,
      'border-width': 0.0,
      'color': '#4579e8',
      'background-image':'data(img)',
      'background-fit':'contain',
      'content':'data(name)',
      'font-size': '70px',
                    },
    'selector': 'node'},
            {'css': {
                'width': 20.0,
            },
            'selector': 'edge'}
            ],
  }]

In [120]:
Cytoscape(data=cytoscape_data(g), visual_style=stobj[0]['style'], layout_name='cose')

Cytoscape(data={'data': [], 'directed': False, 'multigraph': False, 'elements': {'nodes': [{'data': {'img': 'd…